In [1]:
import os ## To get htmls
import re ## to find
import ast
import bs4
import time
import json
import requests
import pandas as pd
import scipy as scipy
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
import matplotlib.pyplot as plt
from webdriver_manager.chrome import ChromeDriverManager

from multiprocessing.pool import ThreadPool
import threading


%matplotlib inline


In [2]:
def get_feelings(soup):

    ### Get feelings
    try:
        feelings = {'Feelings':[],'Negatives':[],'Helps with':[]}

        all_feelings = soup.find_all('div',{"role":'tabpanel'})
        site_effects = soup.find_all('li',{"role":'tab'})

        if len([i.text for i in site_effects]) < 3:
            if "Negatives" not in [i.text for i in site_effects]:
                if "Feelings" in [i.text for i in site_effects]:
                    for i, child in enumerate(all_feelings,0):
                        curr = child.findAll('div',recursive=False)
                        
                        if i == 0:           
                            for item in curr:
                                match = re.findall("[A-Z a-z]+", item.text)[0]
                                feelings['Feelings'].append(match)
                        if i == 1:
                            for item in curr:
                                match = re.findall("[A-Z a-z]+", item.text)[0]
                                feelings['Helps with'].append(match)
                
                else: ## If both Negatives and Feelings don't exist - Fill only the 'Helps with'
                    for i, child in enumerate(all_feelings,0):
                        curr = child.findAll('div',recursive=False)
                        for item in curr:
                            match = re.findall("[A-Z a-z]+", item.text)[0]
                            feelings['Helps with'].append(match)
            
            if "Helps with" not in [i.text for i in site_effects] and len(feelings['Feelings']) == 0 and len(feelings['Negatives']) == 0:

                for i, child in enumerate(all_feelings,0):
                    curr = child.findAll('div',recursive=False)
                    
                    if i == 0:           
                        for item in curr:
                            match = re.findall("[A-Z a-z]+", item.text)[0]
                            feelings['Feelings'].append(match)
                    if i == 1:
                        for item in curr:
                            match = re.findall("[A-Z a-z]+", item.text)[0]
                            feelings['Negatives'].append(match)

            if "Feelings" not in [i.text for i in site_effects] and len(feelings['Negatives']) == 0 and len(feelings['Helps with']) == 0:

                for i, child in enumerate(all_feelings,0):
                    curr = child.findAll('div',recursive=False)
                    
                    if i == 0:           
                        for item in curr:
                            match = re.findall("[A-Z a-z]+", item.text)[0]
                            feelings['Negatives'].append(match)
                    if i == 1:
                        for item in curr:
                            match = re.findall("[A-Z a-z]+", item.text)[0]
                            feelings['Helps with'].append(match)
        
        else:
            for i, child in enumerate(all_feelings,0):
                curr = child.findAll('div',recursive=False)
                
                if i == 0:           
                    for item in curr:
                        match = re.findall("[A-Z a-z]+", item.text)[0]
                        feelings['Feelings'].append(match)
                if i == 1:
                    for item in curr:
                        match = re.findall("[A-Z a-z]+", item.text)[0]
                        feelings['Negatives'].append(match)
                if i == 2:
                    for item in curr:
                        match = re.findall("[A-Z a-z]+", item.text)[0]
                        feelings['Helps with'].append(match)


        if len(feelings['Feelings']) < 5:
            for i in range(5-len(feelings['Feelings'])):
                feelings['Feelings'].append(None) 

        if len(feelings['Negatives']) < 5:
            for i in range(5-len(feelings['Negatives'])):
                feelings['Negatives'].append(None) 

        if len(feelings['Helps with']) < 5:
            for i in range(5-len(feelings['Helps with'])):
                feelings['Helps with'].append(None) 

        return feelings
    
    except Exception as e:
        print(e)
        pass

In [3]:
## Scrape data from html

def get_strainType(soup):

    ## Strain type - Sativa, Indica or Hybrid
    try:
        strain_type = soup.find_all('span',class_='jsx-1167018388 text-xs bg-leafly-white py-sm px-sm rounded')
        if len(strain_type) > 0:
            strain_type = strain_type[0].text
        else:
            strain_type = None

        return strain_type
    except Exception as e:
        print(e)
        pass

def get_terpene(soup):

    ## Top terpene
    try:
        top_terpene = soup.find_all('span',class_='jsx-1167018388 ml-xs')
        if len(top_terpene) > 0:
            top_terpene = top_terpene[0].text
        else:
            top_terpene = None

        return top_terpene
    except Exception as e:
        print(e)
        pass

def get_cannabinoid(soup):
    
    ## Cannabinoids
    try:
        cannabinoids = {"THC": None, "CBD": None, "CBG":None}
        cannabinoids_list = soup.find_all('span',class_='jsx-1167018388 text-xs py-sm px-sm ml-sm rounded')
        if len(cannabinoids_list) > 0:
            for i in soup.find_all('span',class_='jsx-1167018388 text-xs py-sm px-sm ml-sm rounded'):
                if i.text.startswith("THC"):
                    cannabinoids['THC'] = i.text.split()[1]

                elif i.text.startswith("CBD"):
                    cannabinoids['CBD'] = i.text.split()[1]

                elif i.text.startswith("CBG"):
                    cannabinoids['CBG'] = i.text.split()[1]
        else:
            cannabinoids['THC'] = None
            cannabinoids['CBD'] = None
            cannabinoids['CBG'] = None

        return cannabinoids
    except Exception as e:
        print(e)
        pass

def get_rating(soup):

    ## Rating 
    try:
        rating = soup.find_all('span',class_='pr-xs')
        if len(rating) > 0:
            rating = rating[0].text
        else:
            rating = None

        return rating
    except Exception as e:
        print(e)
        pass

def get_ratingNum(soup):

    ## Number of rating users
    try:
        rating_num = soup.find_all('span',class_='pl-xs')
        if len(rating_num) > 0:
            rating_users = re.findall("[0-9]+", rating_num[0].text)[0]
        else:
            rating_users = None

        return rating_users
    except Exception as e:
        print(e)
        pass

def get_name(soup):
    try:
        name = soup.find('h1',{'itemprop':"name"})
        
        if len(name)> 0:
            name = name.text
        else:
            name = None

        return name
    except Exception as e:
        print(e)
        pass


    
    
    



In [4]:
def get_linage(soup):

    left_parent = soup.find_all('div',class_='lineage__left-parent')
    right_parent = soup.find_all('div',class_='lineage__right-parent')
    left_child = soup.find_all('div',class_='lineage__left-child--two-parents')
    right_child = soup.find_all('div',class_='lineage__right-child--two-parents')

    ## Handle parents (might be single parent)
    if len(left_parent) == 0 or len(right_parent) == 0:
        parent = soup.find_all('div','lineage__center-parent')
        if len(parent) > 0:
            left_parent = parent[0].text.split("parent")[0]
            right_parent = 'N/A'
        else:
            left_parent = None
            right_parent = None
    else:
        left_parent = left_parent[0].text.split("parent")[0]
        right_parent = right_parent[0].text.split("parent")[0]

    ## Handle left child (might be child of strain with single parent)
    if len(left_child) == 0:
        left_child = soup.find_all('div',class_='lineage__left-child--single-parent')
        if len(left_child) > 0:
            left_child = left_child[0].text.split("child")[0]
        else:
            left_child = None
    else:
        left_child = left_child[0].text.split("child")[0]

    ## Handle left child (might be child of strain with single parent)
    if len(right_child) == 0:
        right_child = soup.find_all('div',class_='lineage__right-child--single-parent')
        if len(right_child) > 0:
            right_child = right_child[0].text.split("child")[0]
        else:
            right_child = None
    else:
        right_child = right_child[0].text.split("child")[0]

    return left_parent,right_parent,left_child,right_child
    

In [11]:
## Getting all the crawling data

def scrape_data(soup):
    try:
        name = get_name(soup)
        feelings = get_feelings(soup)
        strain_type =  get_strainType(soup)
        cannabinoids =  get_cannabinoid(soup)
        rating = get_rating(soup)
        rating_users = get_ratingNum(soup)
        left_parent,right_parent,left_child,right_child = get_linage(soup)
        top_terpene = get_terpene(soup)
    except Exception as e:
        print(e)
        pass

    return name,feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child,top_terpene

In [6]:
## Get Selenium driver
def get_driver():
    
  driver = getattr(threadLocal, 'driver', None)
  if driver is None:
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(),chrome_options=chromeOptions)
    setattr(threadLocal, 'driver', driver)
  return driver

In [15]:
## Function for the thread to run

def get_strain_data(url):

    full_url = 'https://www.leafly.com' + url
    driver = get_driver()
    driver.get(full_url)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    try:
        name, feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child,top_terpene = scrape_data(soup)
        return name, feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child,top_terpene
    except Exception as e:
        pass
    

In [13]:
def get_strain_names(page_number):

    strains = set()

    url = f'https://www.leafly.com/strains?page={page_number}'
    driver = get_driver()
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source,'html.parser')    
    try: 
        for a in soup.find_all('a',href=True):
            if a['href'].startswith("/strains/") and not a['href'].startswith("/strains/lists"):
                strains.add(a['href'])
    
        return strains

    except Exception as e:
        pass
  


In [16]:

def run_thread(mode,strains=None):
    
    if mode == 'Strain Data':
    
        data = []

        with ThreadPool(10) as pool:
            for name, feelings, strain_type, cannabinoids, rating, rating_users, left_parent,right_parent,left_child,right_child,top_terpene in pool.map(get_strain_data, strains):

                data.append({'Strain Name':name, 
                'Feelings': feelings, 
                'Type': strain_type, 
                'Cannabinoids': cannabinoids, 
                'Rating': rating, 
                'Rating Users': rating_users,
                'Left Parent': left_parent,
                'Right Parent': right_parent,
                'Left Child': left_child,
                'Right Child': right_child,
                'Top Terpene': top_terpene})
        
        return data

    elif mode == 'Strain Names':

        strain_names = []

        url = 'https://www.leafly.com/strains'
        driver = get_driver()
        driver.get(url)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        pages = int(str(soup.find_all('span',class_="mx-lg md:mx-xxl")[0]).split()[-1].split("<")[0])
        max_pages = range(1,pages+1)

        with ThreadPool(10) as pool:
            for result in pool.map(get_strain_names,max_pages):
                strain_names.append(result)


        return strain_names





In [ ]:
threadLocal = threading.local()
start = time.perf_counter()
strain_names = run_thread('Strain Names')
finish = time.perf_counter()
print(f'Finished in {round((finish-start)/60,2)} minutes')








In [ ]:
strain_names_list = list(set().union(*strain_names)) ## Convert list of sets to list of single values

In [ ]:
with open('strain_list.txt','w') as f:
    json.dump(strain_names_list,f)


In [ ]:
with open('strain_list.txt', 'r') as filehandle:
    strains = json.load(filehandle)


print(len(strains))

In [21]:
with open('strain_list.txt', 'r') as filehandle:
    strains = json.load(filehandle)

threadLocal = threading.local()

start = time.perf_counter()

data = run_thread('Strain Data',strains) ## Takes around 25 minutes

finish = time.perf_counter()

print(f'Finished in {round((finish-start)/60,2)} minutes')















====== WebDriver manager ======






====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4

In [22]:
## Create Dataframe

keys = list(data[0].keys())
df = pd.DataFrame(data,columns=keys)

In [23]:
## Save to csv

# df.head()

df.to_csv('canna_df.csv')